# Hopfield Network
A Hopfield network has one layer of $N$ neurons and is a fully interconnected recurrent neural network: each neuron is connected to every other neuron. After initialization of all neurons (the initial input), the network is let to evolve: an output at time $t$ becomes an input at time $t+1$. Thus to generate a series of outputs, we have to provide only one initial input. In the course of this dynamical evolution the network should reach a stable state (an attractor), which is a configuration of neuron values that is not changed by subsequent updates of the network. Networks of this kind are used as models of associative memory. After initialization the network should evolve to the closest attractor.

## Colab Setup
This part is only required when running this notebook "in the cloud" on [Google Colab](https://colab.research.google.com). When running it locally, skip this part and go to the next section.

### [OPTIONAL] Interactive Plots
Run the below cell if you want to enable interactive plots in Google Colab. For most exercises this is not required and it can be quite slow.

In [ ]:
!pip install ipympl
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()

### [REQUIRED] Auxiliary Files

In [ ]:
# Retrieve some auxiliary files from github
!wget https://raw.githubusercontent.com/KULasagna/ANN_DL_public/master/session2/hopfield.py

## Setup
Import all the necessary modules used throughout this notebook and define some methods to visualize the state space of 2D and 3D Hopfield networks, the energy evolution and the handwritten digits.

In [ ]:
# Import the required modules for this notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from keras.datasets import mnist
from hopfield import HopfieldNetwork

rng = np.random.default_rng()

In [ ]:
# 2D and 3D Hopfield networks:
def plot_targets(targets):
  N = targets.shape[1]
  fig, ax = plt.subplots(subplot_kw={'projection': '3d' if N == 3 else None})
  ax.scatter(*np.swapaxes(targets, 0, 1), color='y', marker='D', label='Targets')
  ax.set_xlim([-1.05, 1.05])
  ax.set_ylim([-1.05, 1.05])
  ax.set_zlim([-1.05, 1.05]) if N == 3 else None
  ax.set_title(f'Targets of a {N}D Hopfield network')
  plt.tight_layout()
  plt.show()

def plot_states(states, filename=None):
  if states.ndim == 2:
    states = np.expand_dims(states, 0)
  N = states.shape[1]
  fig, ax = plt.subplots(subplot_kw={'projection': '3d' if N == 3 else None})
  createLineCollection = Line3DCollection if N == 3 else LineCollection
  evolution = createLineCollection(np.swapaxes(states, 1, 2), color='b', zorder=0, label='State Evolution')
  ax.add_collection(evolution)
  s_init = states[:, :, 0]
  s_final = states[:, :, -1]
  ax.scatter(*np.swapaxes(s_init, 0, 1), color='g', marker='*', label='Initial State')
  ax.scatter(*np.swapaxes(s_final, 0, 1), color='r', marker='x', label='Final State')

  ax.set_xlim([-1.05, 1.05])
  ax.set_ylim([-1.05, 1.05])
  ax.set_zlim([-1.05, 1.05]) if N == 3 else None
  ax.set_title(f'Time evolution in the state space of a {N}D Hopfield network')
  ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.06), ncols=4)

  plt.tight_layout()
  if filename is not None:
    plt.savefig(f"{filename}.svg")
  plt.show()

def plot_energies(energies, filename=None):
  if energies.ndim == 1:
    energies = np.expand_dims(energies, 0)
  fig, ax = plt.subplots()
  ax.plot(energies.T)

  ax.set_title('Energy evolution of the states')

  plt.tight_layout()
  if filename is not None:
    plt.savefig(f"{filename}.svg")
  plt.show()

# Digit reconstruction:
def plot_digits(digits, titles=None, filename=None):
  digits = np.asarray(digits)
  C, R = [1, 1, *digits.shape[:-1]][-2:]
  if titles is None:
    titles = [None]*C
  digits = digits.reshape((C, R, 28, 28))
  fig, axes = plt.subplots(R, C, squeeze=False, gridspec_kw={'wspace':1, 'hspace':0.05})
  for c, title in enumerate(titles):
    if title is not None:
      axes[0, c].set_title(title)
    for r in range(R):
      axes[r, c].imshow(digits[c, r], cmap='binary', vmin=-1, vmax=1)
      axes[r, c].axis('off')
  if filename is not None:
    plt.savefig(f'{filename}.svg')
  plt.show()

## 2D Hopfield Network

Create a 2D Hopfield network with target patterns $[1, 1]$, $[-1, -1]$ and $[1, -1]$.

In [ ]:
targets = np.array([[1, 1], [-1, -1], [1, -1]])
net = HopfieldNetwork(targets)
plot_targets(targets)

### **Exercise 1**
Simulate the state evolution for various input vectors (e.g. random points or points of high symmetry) and note down the obtained attractors after a sufficient number of iterations.

In [ ]:
# Random inputs:
inputs = 2*rng.uniform(size=(10, 2))-1  # rng.uniform returns random numbers between 0 and 1, so rescale it to the interval [-1,1]
# Particular inputs of interest:
# inputs = np.array([[0.5, 0.25], [-0.5, -0.1]])  # TODO: Replace these inputs with some points of high symmetry
states, energies = net.simulate(inputs, num_iter=50)
plot_states(states, "hopfield2D")  # Second parameter is the filename of the generated image
plot_energies(energies, "energy")

#### **Exercise 1.a:**
Are the real attractors the same as those used to create the network? If not, why do we get these unwanted attractors?

In [ ]:
# TODO: compare obtained attractors with targets
# Note that the final states (after num_iter steps) can be obtained as:
final_states = states[:, :, -1]

#### **Exercise 1.b:**
How many iterations does it typically take to reach the attractor?

In [ ]:
# TODO: determine number of iterations to reach the attractors

#### **Exercise 1.c:**
What can you say about the stability of the attractors?

In [ ]:
# TODO: examine stability of attractors

## 3D Hopfield Network

Create a 3D Hopfield network with target patterns $[1, 1, 1]$, $[-1, -1, 1]$ and $[1, -1, -1]$.

In [ ]:
targets = np.array([[1, 1, 1], [-1, -1, 1], [1, -1, -1]])
net = HopfieldNetwork(targets)
plot_targets(targets)

### **Exercise 2**
Simulate the state evolution for various input vectors (e.g. random points or points of high symmetry) and note down the obtained attractors after a sufficient number of iterations.

In [ ]:
# Random inputs:
inputs = 2*rng.uniform(size=(10, 3))-1  # rng.uniform returns random numbers between 0 and 1, so rescale it to the interval [-1,1]
# Particular inputs of interest:
# inputs = np.array([[0.5, 0.25, -0.5], [-0.5, -0.1, 0.5]])  # TODO: Replace these inputs with some points of high symmetry
states, energies = net.simulate(inputs, num_iter=50)
plot_states(states, "hopfield3D")  # Second parameter is the filename of the generated image
plot_energies(energies, "energy3D")

#### **Exercise 2.a:**
Are the real attractors the same as those used to create the network? If not, why do we get these unwanted attractors?

In [ ]:
# TODO: compare obtained attractors with targets
# Note that the final states (after num_iter steps) can be obtained as:
final_states = states[:, :, -1]

#### **Exercise 2.b:**
How many iterations does it typically take to reach the attractor?

In [ ]:
# TODO: determine number of iterations to reach the attractors

#### **Exercise 2.c:**
What can you say about the stability of the attractors?

In [ ]:
# TODO: examine stability of attractors

## Digit Reconstruction

We will now train a higher dimensional Hopfield network which has as attractors the handwritten digits from 0 to 9, and test its ability to correctly reconstruct noisy variants of those digits.

We start by loading and preprocessing the MNIST dataset of handwritten digits.

In [ ]:
(digits, labels), _ = mnist.load_data()               # Load MNIST digits dataset
digits = {d: digits[labels == d] for d in range(10)}  # Store digits by class label
targets = [digits[d][0] for d in range(10)]           # Select one digit of each class as a target pattern for the Hopfield network
targets, params = HopfieldNetwork.normalize(targets)  # Normalize the targets (rescaling to the interval [-1, 1])

Create the Hopfield network and visualize its target attractors.

In [ ]:
net = HopfieldNetwork(targets)
plot_digits(targets)

### **Exercise 3**
Add some random noise to the digits and check whether the network is able to recover the original digits. Is the Hopfield model always able to reconstruct the noisy digits? If not, why? What is the influence of the noise on the number of iterations?

Try to answer the above questions by playing with these two parameters.

In [ ]:
# @title { run: "auto" }

# Standard deviation of the added Gaussian noise
noise_level = 1 # @param {type:"slider", min:0, max:10, step:0.1}
# Number of iterations
num_iter = 10 # @param {type:"slider", min:1, max:100, step:1}

In [ ]:
digits_noisy = np.clip(targets + rng.normal(scale=noise_level, size=targets.shape), -1, 1)
states, energies = net.simulate(digits_noisy, num_iter)

In [ ]:
plot_digits([targets, digits_noisy, states[:,:,-1]], ["Attractors", "Noisy Digits", "Reconstructed Digits"], "digits")
plot_energies(energies, "digits_energy")

## Extra
Some additional things you can experiment with (not necessary for the report):


*   The default learning algorithm to setup the Hopfield networks is more complex than the Hebbian rule from the lecture slides. By passing `"Hebb"` as the second parameter when creating the networks, you can see the difference between both learning algorithms.
*   The default simulation algorithm used by the Hopfield networks updates the neurons in synchronous mode. By passing `sync=False` as the third parameter to the `simulate` method, you can experiment with asynchronous updates of the neurons.
*   Instead of adding noise, you can try to pass some alternative digits (not used as training patterns) as initial states to the network; or try to occlude specific parts of the input images.
*   You can experiment with other keras datasets, have a look at the different [available datasets](https://keras.io/api/datasets/).

